In [173]:
import pandas as pd
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, MinMaxScaler,  Imputer, LabelBinarizer, OneHotEncoder,StandardScaler
from sklearn.feature_extraction import DictVectorizer

class LabelEncoderPipelineFriendly(LabelEncoder):
    
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelEncoderPipelineFriendly, self).fit(X)
        
    def transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).transform(X).reshape(-1, 1)

    def fit_transform(self, X, y=None):
        return super(LabelEncoderPipelineFriendly, self).fit(X).transform(X).reshape(-1, 1)

In [174]:
df_train=pd.read_csv('E:\\Kaggle\\dont-overfit-ii\\train.csv')
df_test=pd.read_csv('E:\\Kaggle\\dont-overfit-ii\\test.csv')

In [175]:
#df_train.head()

In [176]:
#df_test.head()

In [177]:
def get_num_cols(df):
    colum_ind=[]
    for i in range(300):
        colum_ind.append(str(i))
    return df[colum_ind]

In [178]:
vec = make_union(*[
    make_pipeline(FunctionTransformer(get_num_cols, validate=False), StandardScaler()),
])

In [179]:
x_train = vec.fit_transform(df_train)
x_test = vec.transform(df_test)
y_train = df_train['target']

In [180]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
# Search through these optino
#C = [0.920,0.921,0.922,0.923,0.94,0.925,0.926,0.927,0.928,0.929]
C=[0.5,0.7,0.9,1.1]
#C=[1.10,1.11,1.12,1.13,1.14,1.15,1.16,1.17,1.18,1.19]
hyperparameters = dict(C=C)

clf = LogisticRegression(solver='liblinear', class_weight='balanced', penalty='l1',random_state=1)
rand_cv = RandomizedSearchCV(clf, hyperparameters, random_state=1, n_iter=1, cv=5, verbose=0, n_jobs=-1, scoring='roc_auc')
best_model = rand_cv.fit(x_train, y_train)

print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best Score: {}'.format(best_model.best_score_))

Best Penalty: l1
Best C: 1.1
Best Score: 0.8131944444444444


In [181]:
y_test=best_model.predict(x_test)

In [182]:
df_predicted = pd.DataFrame({'id': df_test['id'], 'target': y_test})
df_predicted.to_csv('E:\\Kaggle\\dont-overfit-ii\\sample_submission_std.csv', sep=',', index=False)